1. 데이터를 자르니 오히려 점수가 내려감
2. n_estimator 300과 2000의 차이는 0.02 accuracy, 시간은 2분30초 더 걸림
3. lightgbm으로 밀고나가려면 hypterparameter tuning + 전처리에 시간을 투자해야 할 듯
4. bidirectional ltsm도 해보자


In [ ]:
!git clone -- recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
%cd /content/LightGBM/python-package
!sudo python setup.py install — precompile


In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd /content/Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
%cd ../

In [ ]:
import MeCab
import re
import random
import time
import datetime
import numpy as np      
import pandas as pd       

import matplotlib.pyplot as plt   
from IPython.display import Image
from collections import Counter
from tqdm import tqdm 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [ ]:
def preprocess(text):
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    #remove some puncts (except . ! ?)
    text=re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '',text)
    text=" ".join(text.split())
    return text

mecab = MeCab.Tagger()
def mecab_morphs(text):
    morphs = []
    pattern = re.compile(".*\t[A-Z]+") 
    # konlpy의 mecab output 형태와 같게 만들어주기
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    for token in temp:
        morphs.append(token[0])
    return morphs

In [ ]:
%%time
train = pd.read_csv("/content/drive/My Drive/dacon/news_train.csv") # train.csv 불러오기
test = pd.read_csv("/content/drive/My Drive/dacon/news_test.csv") # test.csv 불러오기
train["id"] = train["n_id"].astype(str) + '_' + train["ord"].astype(str)
train.drop(['n_id', 'ord'],axis = 1) 

train['clean_content'] = train['content'].apply(preprocess)
test['clean_content'] = test['content'].apply(preprocess)
train['clean_title'] = train['title'].apply(preprocess)
test['clean_title'] = test['title'].apply(preprocess)

CPU times: user 3.05 s, sys: 140 ms, total: 3.19 s
Wall time: 8.04 s


In [ ]:
test_paragraph=pd.DataFrame(test['clean_content'])
test_title=pd.DataFrame(test['clean_title'])
train_paragraph=pd.DataFrame(train['clean_content'])
train_title=pd.DataFrame(train['clean_title'])
train_yy=pd.DataFrame(train['info'])
test['info']=2
test_yy=pd.DataFrame(test['info'])


In [ ]:
train_par = [(mecab_morphs(x), y) for x, y in tqdm(zip( train_paragraph['clean_content'], train_yy['info']))]
test_par = [(mecab_morphs(x), y) for x, y in tqdm(zip( test_paragraph['clean_content'], test_yy['info']))]
train_tit = [(mecab_morphs(x), y) for x, y in tqdm(zip( test_title['clean_title'], train_yy['info']))]
test_tit = [(mecab_morphs(x), y) for x, y in tqdm(zip( test_title['clean_title'], test_yy['info']))]


118745it [00:16, 7178.62it/s]
142565it [00:19, 7142.53it/s]
118745it [00:07, 15338.51it/s]
142565it [00:09, 14827.03it/s]


In [ ]:
%%time
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
def get_model_input(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    _words = [x for x in _words if x[:-1] not in stopwords]
    for i in range(len(_words)-1):
        yield _words[i]

train_paragraph, Y_train = [], []
for lwords in train_par:
    Y_train.append(lwords[1])
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    train_paragraph.append(" ".join(temp))

test_paragraph = []
for lwords in test_par:
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    test_paragraph.append(" ".join(temp))

train_title = []
for lwords in train_tit:
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    train_title.append(" ".join(temp))

test_title = []
for lwords in test_tit:
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    test_title.append(" ".join(temp))

In [ ]:
%%time
vectorizer=TfidfVectorizer(ngram_range=(1, 3),
    min_df=2,   
    max_features=10000,
    sublinear_tf=True,
    lowercase=False,
    use_idf=True)
vec_train_tit= vectorizer.fit_transform(train_title)
vec_test_tit= vectorizer.transform(test_title)
vec_train_par= vectorizer.fit_transform(train_paragraph)
vec_test_par= vectorizer.transform(test_paragraph)

CPU times: user 14.8 s, sys: 193 ms, total: 15 s
Wall time: 15 s


In [ ]:
Y_train = list(train['info'])

In [ ]:
%%time
#0.95
lgbm = LGBMClassifier(n_estimators=1080,
                     learning_rate=0.058,
                     min_child_samples=85,
                     random_state=2020,
                     )
lgbm.fit(vec_train_par, Y_train)
lgbm_y=lgbm.predict_proba(vec_test_par)
lgbm_result = [1 if i[1] >= 0.5 else 0 for i in lgbm_y]

In [ ]:
result = pd.DataFrame()
result['lgbm'] = lgbm_result

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

def MNB(train_X, train_y, test_X):
    model = CalibratedClassifierCV(MultinomialNB(alpha=0.03), method='isotonic')
    model.fit(train_X, train_y)
    pred_test = model.predict_proba(test_X)
    model_result = [1 if i[1] >= 0.5 else 0 for i in pred_test]
    return model_result

def LinearSVM(train_X, train_y, test_X):
    model = CalibratedClassifierCV(LinearSVC(), method='isotonic')
    model.fit(train_X, train_y)
    pred_test = model.predict_proba(test_X)
    model_result = [1 if i[1] >= 0.5 else 0 for i in pred_test]
    return model_result

In [ ]:
%%time
#0.95
lgbm = LGBMClassifier(n_estimators=1080,
                     learning_rate=0.058,
                     min_child_samples=85,
                     random_state=2020,
                     )
lgbm.fit(vec_train_par, Y_train)
lgbm_y=lgbm.predict_proba(vec_test_par)
lgbm_result = [1 if i[1] >= 0.5 else 0 for i in lgbm_y]
result = pd.dataframe()
result['lgbm'] = lgbm_result

In [ ]:
%%time
result['mnb'] = MNB(vec_train_par, Y_train, vec_test_par)
result['linearsvm'] = LinearSVM(vec_train_par, Y_train, vec_test_par)

CPU times: user 3.83 s, sys: 12 ms, total: 3.84 s
Wall time: 3.85 s


In [ ]:
result

,lgbm,mnb,linearsvm
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
142560,1,1,1
142561,1,1,1
142562,1,1,1
142563,1,1,1


In [ ]:
result['mean'] = result['lgbm']*0.9+result['mnb']*0.1+result['linearsvm']*0.1

In [ ]:
%%time
submission = pd.DataFrame(
{"id" : test.id,
"info" : result['mean']})
submission.head()
submission.to_csv("mecab_lightGBM_param8.csv", index = False)

CPU times: user 241 ms, sys: 6.99 ms, total: 248 ms
Wall time: 248 ms
